In [ ]:
from newspaper import Article
import random
import string
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import nltk
import numpy as np
import warnings
import pyttsx3 
import speech_recognition as sr 
import datetime
import os
import smtplib


warnings.filterwarnings('ignore')

nltk.download('punkt', quiet=True)
nltk.download('wordnet', quiet=True)

engine = pyttsx3.init('sapi5')
voices = engine.getProperty('voices')
# print(voices[1].id)
engine.setProperty('voice', voices[0].id)


def speak(audio):
    engine.say(audio)
    engine.runAndWait()




article = Article('https://www.mayoclinic.org/diseases-conditions/coronavirus/symptoms-causes/syc-20479963?page=0&citems=10')
article.download()
article.parse()
article.nlp()
corpus = article.text


text = corpus
sent_tokens = nltk.sent_tokenize(text)


remove_punct_dict = dict(  ( ord(punct),None) for punct in string.punctuation)



def LemNormalize(text):
  return nltk.word_tokenize(text.lower().translate(remove_punct_dict))


GREETING_INPUTS = ["hi", "hello", "hola", "greetings", "wassup", "hey"]

GREETING_RESPONSES=["howdy", "hi", "hey", "what's good", "hello", "hey there"]

def greeting(sentence):
    for word in sentence.split():
        if word.lower() in GREETING_INPUTS:
            return random.choice(GREETING_RESPONSES)


def response(user_response):

    user_response = user_response.lower()  

    robo_response = ''

    sent_tokens.append(user_response)

    TfidfVec = TfidfVectorizer(tokenizer=LemNormalize, stop_words='english')

    tfidf = TfidfVec.fit_transform(sent_tokens)

    vals = cosine_similarity(tfidf[-1], tfidf)

    idx = vals.argsort()[0][-2]

    flat = vals.flatten()

    flat.sort()
    score = flat[-2]

    if (score == 0):
        robo_response = robo_response + "I apologize, I don't understand."
    else:
        robo_response = robo_response + sent_tokens[idx]

    sent_tokens.remove(user_response)

    return robo_response
flag = True
print("Voicechatbot: I am Voicechatbot and I will answer your queries about Coronavirus. If you want to exit, say thanks or thank you!")
a="I am Voicechatbot and I will answer your queries about Coronavirus. If you want to exit, say thanks or thank you!"
speak(a)
while(flag == True):
    r = sr.Recognizer()
    with sr.Microphone() as source:

        r.pause_threshold = .75
        audio = r.listen(source)

    try:

        query = r.recognize_google(audio, language='en-in')
        print(f"User said: {query}\n")

    except Exception as e:
        # print(e)

        query=""

    user_response = query
    user_response = user_response.lower()
    if(user_response != 'exit'):
      if(user_response == 'thanks' or user_response =='thank you'):
        flag=False
        print("Voicechatbot: You are welcome !")
        a=" You are welcome !"
        speak(a)
      else:
        if(greeting(user_response) != None):
          print("Voicechatbot: "+greeting(user_response))
          a=greeting(user_response)
          print(a)
          speak(a)

        elif user_response=="how are you":
            print("Voicechatbot: I am good, tell me how can I help you ")
            a="I am good, tell me how can I help you "
            speak(a)
        else:
          print("Voicechatbot: "+response(user_response))
          a = response(user_response)
          speak(a)
    else:
      flag = False
      print("Voicechatbot: Chat with you later !")
      a="Chat with you later !"
      speak(a)


Voicechatbot: I am Voicechatbot and I will answer your queries about Coronavirus. If you want to exit, say thanks or thank you!
Voicechatbot: I apologize, I don't understand.
User said: hello

Voicechatbot: hi
hey
User said: Corona virus

Voicechatbot: The virus spreads by respiratory droplets released when someone with the virus coughs, sneezes or talks.
User said: interns

Voicechatbot: I apologize, I don't understand.
